In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [8]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [25]:
# add your code here - consider creating a new cell for each section of code

### Removing null values

In [ ]:
# Removing null values

df_books.isnull().sum()
df_books.dropna(inplace=True)

df_ratings.isnull().sum()
df_books.isnull().sum()

### Removing Users with less than 200 ratings & books with less than 100 ratings

In [97]:

# Group by 'user' and 'isbn' columns and count the number of ratings per user and book
user_counts = df_ratings.groupby('user')['rating'].count()
book_counts = df_ratings.groupby('isbn')['rating'].count()

# Filter out users with less than 200 ratings and books with less than 100 ratings
df_ratings_filtered = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 200].index)]
df_books_filtered = df_ratings_filtered[df_ratings_filtered['isbn'].isin(book_counts[book_counts >= 100].index)]


### Preparing Data for the Model

In [100]:
# Merge the books and ratings dataframes
merged_df = pd.merge(df_books_filtered, df_books, on='isbn')

# Remove unnecessary columns
merged_df = merged_df[['title', 'isbn', 'user', 'rating']]

# Reshape data into matrix format for NearestNeighbors model
ratings_matrix = merged_df.pivot_table(index='title', columns='user', values='rating').fillna(0)

ratings_matrix.head()


user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Train NearestNeighbors model on ratings matrix

In [150]:
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(ratings_matrix.values)

NearestNeighbors(algorithm='brute', metric='cosine')

In [153]:
def get_recommends(book_title):

    # Get the rating vector for the input book title
    book_vector = ratings_matrix.loc[book_title].values.reshape(1, -1)

    # Get the distances and indices of the 6 nearest neighbors to the input book
    distances, indices = model.kneighbors(book_vector, n_neighbors=6)

    # Get the titles and distances of the nearest books
    recommended_books = []
    for i in range(1, len(distances.flatten())):
        recommended_books.append(
            [
                ratings_matrix.index[indices.flatten()[i]],
                distances.flatten()[i]
            ]
        )

    # Sort the recommended books in ascending order based on their distance and reverse the list
    recommended_books = sorted(recommended_books, key=lambda x: x[1])
    recommended_books.reverse()

    # Return the recommendations in the expected format if there are at least 2 recommended books
    if len(recommended_books) >= 2:
        recommended_books_list = []
        recommended_books_list.append(book_title)
        recommended_books_list.append(recommended_books)
        return recommended_books_list

    # Return an empty list if there are less than 2 recommended books
    else:
        return []
    


In [152]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You havn't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
